# Подготовка данных для OCR

In [1]:
import sys
import os
import pathlib
import shutil
import requests
from pdf2image import convert_from_path
import pandas as pd

In [2]:
SERVERE_DATA_PATH = 'smb://rosa_srv21'
PATH = '/home/kennykey/datasets/OCR/ocr_test/test_1/'
SAVE_IMG_PATH = PATH + 'jpg/'
PREPEARED_IMG_PATH = '/home/kennykey/datasets/OCR/ocr_test/test_1/prepeared_jpg/'
SAMPLE_PATH = PATH + '601753398.pdf'

In [4]:
files = os.listdir(path=PATH)

In [8]:
for file in files:
    if file.endswith('.pdf'):
        pages = convert_from_path(PATH + file, dpi=500)
        file_name = file.split('.')[0]
        if len(pages) == 1:
            for i, page in enumerate(pages):
                fname = SAVE_IMG_PATH + file_name + '.jpg'
                page.save(SAVE_IMG_PATH + file_name + '.jpg')
        if len(pages) > 1:
            for i, page in enumerate(pages):
                fname = SAVE_IMG_PATH + file_name + '_' + str(i) + '.jpg'
                page.save(fname, "JPEG")

## Подготовка .csv

In [54]:
df = pd.read_csv('/home/kennykey/datasets/OCR/ocr_test/prepeared_jpg/ocr_test_dataset.csv', sep='\t')

In [55]:
df['file_name'] = df['file_name'].apply(lambda x: x.split('/')[-1])

In [56]:
df.head()

,file_name,areas,doc_number,scan_is_good,barcode_num
0,001.jpg,"{""shape"":""rectangle""\,""left"":0.070738465218249...",??????????,0,286??????7?11
1,002.jpg,"{""shape"":""rectangle""\,""left"":0.085597602505985...",6017537341,1,2860610860917
2,003.jpg,"{""shape"":""rectangle""\,""left"":0.072787692180747...",6002548256,1,2860611441504
3,004.jpg,"{""shape"":""rectangle""\,""left"":0.077537764212381...",6002548658,1,2860613107354
4,005.jpg,"{""shape"":""rectangle""\,""left"":0.071783172845108...",6002548937,1,2860611888071


In [66]:
df['doc_number'].value_counts().head(15)

6017533986    3
??????????    2
6600245149    2
6017266229    2
6600002416    2
??1753?136    2
6017645767    2
6600275900    2
6017268051    2
6017533721    2
??17???744    2
6017537341    2
??1754??11    1
6017540704    1
6017559651    1
Name: doc_number, dtype: int64

In [68]:
uniq_docs = df['doc_number'].value_counts()

In [71]:
uniq_docs[uniq_docs > 1]

6017533986    3
??????????    2
6600245149    2
6017266229    2
6600002416    2
??1753?136    2
6017645767    2
6600275900    2
6017268051    2
6017533721    2
??17???744    2
6017537341    2
Name: doc_number, dtype: int64

In [73]:
uniq_docs[uniq_docs > 1].sum()

25

In [82]:
df = df.drop_duplicates(subset=['doc_number', 'barcode_num'], keep='first')

In [83]:
df['doc_number'].value_counts()

??????????    2
6017645767    2
??1753?136    2
??17???744    2
6017647375    1
             ..
6017410823    1
6017407243    1
6017269311    1
6017268051    1
6017415830    1
Name: doc_number, Length: 87, dtype: int64

In [103]:
df['barcode_num'][88] = '?????????29??'

In [104]:
df = df.drop_duplicates(subset=['doc_number', 'barcode_num'], keep='last')

In [105]:
df['doc_number'].value_counts()

??????????    2
6017559651    1
6017648400    1
6017647375    1
6017644991    1
             ..
6017410823    1
6017407243    1
6017269311    1
6017268051    1
6017415830    1
Name: doc_number, Length: 87, dtype: int64

In [106]:
df[df['doc_number'] == df['doc_number'].value_counts().index[1]]

,file_name,areas,doc_number,scan_is_good,barcode_num
60,061.jpg,"{""shape"":""rectangle""\,""left"":0.066036589952463...",6017559651,1,2860614792009


In [108]:
df = df.reset_index(drop=True)

In [110]:
df.head()

,file_name,areas,doc_number,scan_is_good,barcode_num
0,001.jpg,"{""shape"":""rectangle""\,""left"":0.070738465218249...",??????????,0,286??????7?11
1,002.jpg,"{""shape"":""rectangle""\,""left"":0.085597602505985...",6017537341,1,2860610860917
2,003.jpg,"{""shape"":""rectangle""\,""left"":0.072787692180747...",6002548256,1,2860611441504
3,004.jpg,"{""shape"":""rectangle""\,""left"":0.077537764212381...",6002548658,1,2860613107354
4,005.jpg,"{""shape"":""rectangle""\,""left"":0.071783172845108...",6002548937,1,2860611888071


In [111]:
uniq_files_list = list(df['file_name'])

In [116]:
prepeared_images = os.listdir(PREPEARED_IMG_PATH)

In [117]:
for image in prepeared_images:
    if image not in uniq_files_list:
        os.remove(PREPEARED_IMG_PATH + image)

In [119]:
df.to_csv(PREPEARED_IMG_PATH + 'ocr_test_dataset_ready.csv')